# YouTube Corpus videos downloader 2023

## Libraries 

In [1]:
import pandas as pd
from IPython.display import clear_output
from pytube import YouTube
from pytube.exceptions import VideoUnavailable
import os
import time 
from tinydb import TinyDB, Query

Variables, funciones y constantes globales útiles en varias partes del código

In [2]:
audios_dir = os.path.join(os.getcwd(), "audios")


ESTADOS = {1:"Aguascalientes", 2:"Baja California", 3:"Baja California Sur", 4:"Campeche", 
            5:"Chiapas", 6:"Chihuahua", 7:"Coahuila", 8:"Colima", 9:"Durango", 10:"Guanajuato",
            11:"Guerrero", 12:"Hidalgo", 13:"Jalisco", 14:"Ciudad de México", 15:"Estado de México", 
            16:"Michoacán", 17:"Morelos", 18:"Nayarit", 19:"Nuevo León", 20:"Oaxaca", 21:"Puebla", 22:"Querétaro", 
            23:"Quintana Roo", 24:"San Luis Potosí", 25:"Sinaloa", 26:"Sonora", 27:"Tabasco", 28:"Tamaulipas",
            29:"Tlaxcala", 30:"Veracruz", 31:"Yucatán", 32:"Zacatecas"}

def clear_terminal():
    # Clear the terminal screen based on the operating system
    if os.name == 'posix':  # Unix/Linux/Mac OS
        os.system('clear')
    elif os.name == 'nt':   # Windows
        os.system('cls')

To do:
- subir audios a una bd para no descargar de yt (maybe)
- crear queries de busqueda compuesta !?
- usar regex para facilitar busqueda por autor
- generar transcripcion por whisper
- volver a generar db, pero ahora que ligue a la ubicacion de la transcripcion de yt y whisper.
- migrar a pytubefix 

## Create dataframe

In [3]:
df = pd.read_csv("corpus_yt_2023.csv")
df.rename(str.strip, axis="columns", inplace = True)
df.loc[:, "Nombre"].ffill(limit=2, inplace = True)
df.loc[:, "Estado"].ffill(inplace=True)
df.loc[:, "Edad"].ffill(limit=2, inplace = True)
df.loc[:, "Género"].ffill(limit=2, inplace = True)

df.head(20)

,Estado,Nombre,Edad,Género,Título,Fecha de creación,Link,Fecha de recopilación,Aportación
0,Aguascalientes,Alejandra García,26.0,Femenino,REGALO COMÍDA en la CALLE | lo logramos 🙏🏻🥳 Al...,31 agosto 2023,https://youtu.be/hWKy7Ns1pNo?si=CtYGM3G0f_TtUXUz,18 septiembre 2023,Heili
1,Aguascalientes,Alejandra García,26.0,Femenino,"habitación principal | REMODELACIÓN, CON POQUI...",30 noviembre 2022,https://youtu.be/bm2xz0_HrGc?si=anegbcnciRTpv535,18 septiembre 2023,Heili
2,Aguascalientes,Alejandra García,26.0,Femenino,Nuestro Árbol de Navidad Casero | Ale García ✨,21 noviembre 2021,https://youtu.be/JT3WoDrrl1A?si=U7yvebArA8Qx6wkv,18 septiembre 2023,Heili
3,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Database 

In [4]:
db_file = "audio_database.json"
DOWNLOADED = [] # rows of the audios that are already downloaded

try:
    with open(db_file, 'r'):
        pass
    db = TinyDB(db_file)
except FileNotFoundError:
    db = TinyDB(db_file)


# Guardamos: video_id, estado, autor, edad, genero, ruta
def save_audio_index(index_downloaded):
    with open("downloaded.txt", 'a') as file:
        file.write(str(index_downloaded) + '\n')

def create_db_entries(df_entry, r):
    id = YouTube(df_entry["Link"]).video_id
    db.insert({"Estado":df_entry["Estado"], "Autor":df_entry["Nombre"], "Edad":df_entry["Edad"], 
               "Genero":df_entry["Género"], "VideoID":id, "Ruta":r})

def load_downloaded_audios():
    # archivo txt con los indices de los auidos ya descargados
    try:
        with open("downloaded.txt", "r") as file:
            for row_num in file:
                val = int(row_num.strip("\n"))
                DOWNLOADED.append(val)
    except FileNotFoundError as e:
        print(f"File not found: {e}")

def errase_db():
    db.truncate()
    db.all()

def errase_downloaded_txt():
    try:
        with open("downloaded.txt", "w") as _:
           pass
    except FileNotFoundError as e:
        print(f'File not found: {e}')

## Download audios

In [5]:
def download_audio(link, row, i):
    
    try: 
        yt_obj = YouTube(link)
    except VideoUnavailable:
        print(f'Video at {link} is unavailable, skipping...')
    else:
        try:
            audio = yt_obj.streams.last()
            file_name = f'{yt_obj.video_id}.webm'
            path = os.path.join(audios_dir, file_name)
            audio.download(filename=file_name, output_path=audios_dir, timeout=60)
            save_audio_index(i) 
            create_db_entries(row, path)
            print(f'Downloaded {yt_obj.title[0:20] + "..."}')
        except Exception as e:
            print(f'An error occurred while downloading the video: {str(e)}')


## Main

In [6]:
downloaded_counter = 0 

df_available = df.dropna()
available_total = len(df_available)
load_downloaded_audios()

for i, row in df_available.iterrows():

    link = row["Link"]

    if downloaded_counter % 5 == 0 and downloaded_counter != 0: 
        # Cada 5 descargas esperamos un minuto para evitar ser bloqueados
        time.sleep(60)

    if i in DOWNLOADED:
        print("Already downloaded. Skiping...")
        continue
    else:
        download_audio(link, row, i)

    downloaded_counter += 1


print(f"\n\nFinished downloading. {downloaded_counter} of {available_total} videos downloaded.")

Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
An error occurred while downloading the video: QbRt-eZwuno is age restricted, and can't be accessed without logging in.
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already down

In [7]:
#errase_db()
#errase_downloaded_txt()

In [5]:
len(db.all())

65

# Busqueda segun atributos

In [6]:
#### DIFERENTES QUERIES

def state_equals(estado, return_results=True) :
    Video = Query()
    if return_results:
        return db.search(Video.Estado == estado)
    else:
        return Video.Estado == estado

def age_equals(edad, return_results=True):
    Video = Query()
    if return_results:
        return db.search(Video.Edad == edad)
    else:
        return Video.Edad == edad

def age_is_more(num=18, return_results=True):
    # Busca por edad que sea mayor o igual a un numero 
    Video = Query()
    if return_results:
        return db.search(Video.Edad >= num)
    else:
        return Video.Edad >= num

def age_is_less(num=18, return_results=True):
    Video = Query()
    if return_results:
        return db.search(Video.Edad <= num)
    else:
        Video.Edad <= num

def gender_is(genero, return_results=True):
    # Busca por genero, feminino o masculino
    Video = Query()
    if return_results:
        return db.search(Video.Genero == genero)
    else:
        return Video.Genero == genero

def autor_is(autor, return_results=True):
    ## Agregar regex
    Video = Query()
    if return_results:
        return db.search(Video.Nombre == autor)
    else:
        return Video.Nombre == autor



In [7]:
print(state_equals("Baja California"))
print(age_equals(36))
print(age_is_more(18))
print(age_is_less(50))
print(gender_is("Masculino"))


[{'Estado': 'Baja California', 'Autor': 'Carolina Vázquez Díaz', 'Edad': 29.0, 'Genero': 'Femenino', 'VideoID': 'H9zhwMHfAzA', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\H9zhwMHfAzA.webm'}, {'Estado': 'Baja California', 'Autor': 'Carolina Vázquez Díaz', 'Edad': 29.0, 'Genero': 'Femenino', 'VideoID': 'kS_in6b-nxA', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\kS_in6b-nxA.webm'}, {'Estado': 'Baja California', 'Autor': 'Carolina Vázquez Díaz', 'Edad': 29.0, 'Genero': 'Femenino', 'VideoID': 'MtIPR7anf98', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\MtIPR7anf98.webm'}, {'Estado': 'Baja California', 'Autor': 'Kimberly Shantal Adame', 'Edad': 30.0, 'Genero': 'Femenino', 'VideoID': 'QvcnM_Yx1q0', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\QvcnM_Yx1q0.webm'}, {'Esta

## Conseguir la ruta de los audios segun ciertos atributos
-----
Para poder ubicar la ruta de los audios según un criterio de búsqueda, considera los siguientes atributos de búsqueda: 

1. **Estado**: entidad federativa del hablante.
2. **Autor**: nombre del hablante. 
3. **Edad**: edad del hablante.
4. **Género**: género del hablante (masculino o femenino).

### Tipos de búsqueda
#### 1. Búsqueda simple
Se utiliza cuando se desea hacer una búsqueda utilizando un solo atributo, por ejemplo: 
- Encontrar audios del **Estado** de Baja California.
- Buscar todos los audios cuya **edad** del hablante sea mayor o igual que 18. 
- Conseguir todos los audios del **género** masculino.

#### 2. Búsqueda compuesta
Se utiliza cuando se desea realizar una búsqueda combinando varios atributos, por ejemplo: 
- Encontrar audios del **estado** de Baja California cuyos hablantes sean del **género** masculino.


El siguiente bloque de código pregunta por estos dos tipos de búsqueda para poder filtrar la búsqueda

## Cómo usarlo
----
Ejecuta el código. Selecciona 1 si deseas realizar una búsqueda simple. 2 si deseas una compuesta. 
1. **Busqueda simple (un atributo)**
    - Selecciona el número del atributo que desees consultar. 
    - Unas opciones se presentarán según el criterio a buscar. Brindar la información necesaria. 
    - Se muestran los resultados correspondientes
2. **Busqueda compuesta (varios atributos)**
    - Escoge el número de atributos. Considera que para el estado y autor pueden seleccionarse varios, se consideran atributos diferentes (estado igual a Baja California y estado igual a Colima se consideran 2 atributos).

In [9]:
### IMPRIMIR RUTAS SEGUN MATCH ###

def print_opciones():
    print("Selecciona un criterio de búsqueda: ")
    print("1. Estado\n2. Autor\n3. Edad\n4. Género")

def imprimir_edos():
    for estado in ESTADOS.items():
        print(estado)

def seleccionar_edad(return_results = True) -> Query:

    edad = int(input("Ingresa la edad de búsqueda: "))
    print("Ingresa el número según tu criterio de búsqueda.")
    print(f"1. Edad igual a {edad}")
    print(f"2. Edad mayor o igual a {edad}")
    print(f"3. Edad menor o igual a {edad}")

    op = int(input())

    if op == 1:
        clear_output()
        print(f"Buscando resultados con edad igual a: {edad}")
        if return_results:
            return age_equals(edad) 
        else:
            return age_equals(edad, False)
    elif op == 2:
        clear_output()
        print(f"Buscando resultados con edad mayor o igual a: {edad}")
        if return_results:
            return age_is_more(edad) 
        else:
            return age_is_more(edad, False)
    elif op == 3:
        clear_output()
        print(f"Buscando resultados con edad menor o igual a: {edad}")
        if return_results:
            return age_is_less(edad) 
        else:
            return age_is_less(edad, False)
    else:
        print("Opción inválida")
    
    return []

def salvar_resultados(query):
    print(f"\nSalvando resultados en: {os.getcwd()}")
    current_time = time.time()
    struct = time.localtime(current_time)
    s = time.strftime("%H-%M-%S", struct)
    print(s)
    with open(f"{s}.txt", "w") as f:
        for item in query:
            f.write(str(item))
            f.write("\n")

def escoger_opciones(opcion: int) -> list:
    
    resultados = ""

    if opcion == 1: 
        imprimir_edos()
        seleccion = int(input("Escribe el número de Estado a buscar: "))
        clear_output()
        print(f"Buscando resultados con estado igual a: {ESTADOS[seleccion]}")
        time.sleep(3)
        print("*")
        resultados = state_equals(ESTADOS[seleccion])
        print(resultados)

    elif opcion == 2: 
        seleccion = input("Escribe el nombre del autor a buscar. Incluye acentos: ")
        print(f"Buscando resultados con autor igual a: {seleccion}")
        time.sleep(3)
        print("*")
        resultados = autor_is(seleccion)
        print(resultados)

    elif opcion == 3: 
        resultados = seleccionar_edad()
        time.sleep(3)
        print("*")
        print(resultados)

    elif opcion == 4:
        print("Escoge el genero a buscar, ingresando el número:\n1. Masculino\n2. Femenino")
        op = int(input(" "))
        seleccion = 'Masculino' if op == 1 else 'Femenino'
        clear_output()
        print(f"Buscando resultados con género igual a: {seleccion}")
        time.sleep(3)
        print("*")
        resultados = gender_is(seleccion)
        print(resultados)
        
    else: 
        print("Seleccion invalida.")

    return resultados


#######################################################

query_finished = False

while not query_finished:
    tipo_busqueda = int(input("Ingresa el tipo de busqueda: "))
    if tipo_busqueda == 1: 
        print_opciones()
        op = int(input("Escribe el número de la opción a escoger: "))
        resultados_final = escoger_opciones(op)
        query_finished = True
    elif tipo_busqueda == 2:
        n = int(input("\n¿Cuántos atributos tendrá tu consulta?"))
        atributos_elegidos = []
        Video = Query()
        query_compuesta = Query()
        ### SOLO OPERACIONES AND ###
        for i in range(n):
            print_opciones()
            at = int(input(f"{i + 1}. Ingresa el número del atributo deseado"))
            if at == 1:
                imprimir_edos()
                edo = int(input("Ingresa el número del estado: "))
                qn = state_equals(ESTADOS[edo], False) # Dando false, nos regresa el query. No resultados.
                atributos_elegidos.append('Estado')
                query_compuesta = query_compuesta & qn 
                time.sleep(1)
            elif at == 2: 
                nom = input("Ingresa el nombre del autor: ")
                qn = autor_is(nom, False) # Dando false, nos regresa el query. No resultados.
                atributos_elegidos.append('Autor')
                query_compuesta = query_compuesta & qn 
                time.sleep(1)
            elif at == 3: 
                qn = seleccionar_edad(False)
                query_compuesta = query_compuesta & qn
                atributos_elegidos.append('Edad')
                time.sleep(1)
            elif at == 4:
                print("Escoge el genero a buscar, ingresando el número:\n1. Masculino\n2. Femenino")
                op = int(input(" "))
                seleccion = 'Masculino' if op == 1 else 'Femenino'
                clear_output()
                qn = gender_is(seleccion, False)
                query_compuesta = query_compuesta & qn
                atributos_elegidos.append('Genero')
                time.sleep(1)
            else:
                print("Opcion invalida")
                break

            clear_output()
        # Ya tenemos los atributos 
       
        query_finished = True
    else:
        print("Selecciona una opción válida.")
        query_finished = True

if tipo_busqueda == 2 and query_finished:
    print(query_compuesta)
    resultados_final = db.search(query_compuesta)
    print(resultados_final)

if query_finished:
    salvar = int(input("Deseas guardar la búsqueda en un archivo txt? 1 - Sí. 2 - No."))
    if salvar == 1:
        salvar_resultados(resultados_final)

Buscando resultados con género igual a: Femenino
*
[{'Estado': 'Aguascalientes', 'Autor': 'Alejandra García ', 'Edad': 26.0, 'Genero': 'Femenino', 'VideoID': 'hWKy7Ns1pNo', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\hWKy7Ns1pNo.webm'}, {'Estado': 'Aguascalientes', 'Autor': 'Alejandra García ', 'Edad': 26.0, 'Genero': 'Femenino', 'VideoID': 'bm2xz0_HrGc', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\bm2xz0_HrGc.webm'}, {'Estado': 'Aguascalientes', 'Autor': 'Alejandra García ', 'Edad': 26.0, 'Genero': 'Femenino', 'VideoID': 'JT3WoDrrl1A', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\JT3WoDrrl1A.webm'}, {'Estado': 'Baja California', 'Autor': 'Carolina Vázquez Díaz', 'Edad': 29.0, 'Genero': 'Femenino', 'VideoID': 'H9zhwMHfAzA', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\